In [1]:
import gym
import numpy as np
import operator
from IPython.display import clear_output
from time import sleep
import random
import itertools
import tqdm

tqdm.monitor_interval = 0

In [2]:
def create_random_policy(env):
    policy = {}
    for key in range(0, env.observation_space.n):
        current_end = 0
        p = {}
        for action in range(0, env.action_space.n):
            p[action] = 1 / env.action_space.n
        policy[key] = p
    return policy

In [3]:
def create_state_action_dictionary(env, policy):
    Q = {}
    for key in policy.keys():
         Q[key] = {a: 0.0 for a in range(0, env.action_space.n)}
    return Q

In [4]:
def run_game(env, policy, display=True):
    env.reset()
    episode = []
    finished = False

    while not finished:
        s = env.env.s
        if display:
            clear_output(True)
            env.render()
            sleep(1)

        timestep = []
        timestep.append(s)
        n = random.uniform(0, sum(policy[s].values()))
        top_range = 0
        for prob in policy[s].items():
            top_range += prob[1]
            if n < top_range:
                action = prob[0]
                break 
        state, reward, finished, info = env.step(action)
        timestep.append(action)
        timestep.append(reward)

        episode.append(timestep)

    if display:
        clear_output(True)
        env.render()
        sleep(1)
    return episode

In [5]:
def test_policy(policy, env):
    wins = 0
    r = 100
    for i in range(r):
        w = run_game(env, policy, display=False)[-1][-1]
        if w == 1:
            wins += 1
    return wins / r

In [42]:
def monte_carlo_e_soft(env, episodes=100, policy=None, epsilon=0.01):
    if not policy:
        policy = create_random_policy(env)  # Create an empty dictionary to store state action values    
    Q = create_state_action_dictionary(env, policy) # Empty dictionary for storing rewards for each state-action pair
    returns = {} # 3.
    
    for _ in range(episodes): # Looping through episodes
        G = 0 # Store cumulative reward in G (initialized at 0)
        episode = run_game(env=env, policy=policy, display=False) # Store state, action and value respectively 
        
        # for loop through reversed indices of episode array. 
        # The logic behind it being reversed is that the eventual reward would be at the end. 
        # So we have to go back from the last timestep to the first one propagating result from the future.
        
        for i in reversed(range(0, len(episode))):   
            s_t, a_t, r_t = episode[i] 
            state_action = (s_t, a_t)
            G += r_t # Increment total reward by reward on current timestep
            
            if not state_action in [(x[0], x[1]) for x in episode[0:i]]: # 
                if returns.get(state_action):
                    returns[state_action].append(G)
                else:
                    returns[state_action] = [G]   
                    
                Q[s_t][a_t] = sum(returns[state_action]) / len(returns[state_action]) # Average reward across episodes
                
                Q_list = list(map(lambda x: x[1], Q[s_t].items())) # Finding the action with maximum value
                indices = [i for i, x in enumerate(Q_list) if x == max(Q_list)]
                max_Q = random.choice(indices)
                
                A_star = max_Q # 14.
                
                for a in policy[s_t].items(): # Update action probability for s_t in policy
                    if a[0] == A_star:
                        policy[s_t][a[0]] = 1 - epsilon + (epsilon / abs(sum(policy[s_t].values())))
                    else:
                        policy[s_t][a[0]] = (epsilon / abs(sum(policy[s_t].values())))

    return policy

In [43]:
env = gym.make('Taxi-v3')

In [44]:
policy = monte_carlo_e_soft(env, episodes=500000)

In [45]:
policy[0]

{0: 0.16666666666666666,
 1: 0.16666666666666666,
 2: 0.16666666666666666,
 3: 0.16666666666666666,
 4: 0.16666666666666666,
 5: 0.16666666666666666}

In [46]:
policy_mc=[]
for k in policy.keys():
    acc= max(policy[k].items(), key=lambda k : k[1])[0]
    policy_mc.append(acc)

In [47]:
policy_mc= np.array(policy_mc)

In [50]:
action = env.reset()
done = False
final_reward=0
while done==False:
    action=policy_mc[action]
    action, reward, done, _ = env.step(action)
    final_reward = final_reward + reward
    env.render()
    sleep(1)
print(final_reward)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (East)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (North)
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Pickup)
+---------+
|R: | : :G|
| : | : :_|
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (South)
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y|

In [49]:
policy_mc

array([0, 4, 3, 4, 2, 0, 3, 2, 0, 0, 0, 0, 0, 0, 1, 0, 5, 5, 5, 2, 0, 0,
       0, 3, 0, 0, 2, 0, 3, 0, 0, 0, 0, 3, 3, 0, 3, 3, 0, 0, 0, 1, 1, 0,
       2, 0, 3, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 5, 0, 3, 0, 2, 0, 3, 2, 0,
       2, 2, 3, 1, 0, 0, 3, 2, 0, 0, 3, 2, 2, 2, 0, 1, 1, 3, 4, 0, 0, 4,
       3, 3, 0, 3, 3, 0, 1, 0, 0, 5, 5, 0, 0, 2, 2, 1, 3, 0, 1, 0, 0, 0,
       0, 2, 2, 0, 0, 0, 1, 2, 2, 2, 0, 3, 3, 1, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 3, 0, 2, 0, 2, 2, 0, 3, 0, 0, 0, 2,
       3, 0, 0, 0, 0, 0, 0, 3, 0, 3, 2, 0, 1, 1, 3, 2, 0, 3, 1, 0, 0, 0,
       3, 0, 2, 0, 0, 0, 2, 3, 1, 0, 3, 1, 1, 0, 0, 3, 3, 3, 1, 0, 3, 1,
       0, 3, 0, 1, 2, 1, 2, 0, 3, 2, 0, 3, 0, 0, 1, 2, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 3, 2, 0, 2, 2, 3, 2, 0, 3, 2, 0, 0, 0, 3, 3, 2, 2, 0, 0,
       3, 0, 1, 0, 1, 1, 3, 3, 0, 3, 2, 1, 3, 0, 3, 2, 2, 2, 0, 1, 2, 2,
       1, 0, 1, 1, 3, 1, 0, 3, 0, 0, 2, 0, 1, 2, 2, 0, 0, 2, 1, 1, 3, 0,
       0, 3, 3, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0,